## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kuminskiy,RU,58.6667,66.5667,33.06,76,94,7.58,overcast clouds
1,1,Kapaa,US,22.0752,-159.3190,82.72,72,90,5.01,light rain
2,2,Zaliztsi,UA,49.7919,25.3730,44.74,58,7,10.27,clear sky
3,3,Atuona,PF,-9.8000,-139.0333,78.28,74,19,20.42,few clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,75.16,89,89,9.24,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                        (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kapaa,US,22.0752,-159.3190,82.72,72,90,5.01,light rain
3,3,Atuona,PF,-9.8000,-139.0333,78.28,74,19,20.42,few clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,75.16,89,89,9.24,overcast clouds
5,5,Vaini,TO,-21.2000,-175.2000,77.16,83,75,12.66,broken clouds
6,6,Rikitea,PF,-23.1203,-134.9692,73.11,68,24,21.39,few clouds
7,7,Hermanus,ZA,-34.4187,19.2345,51.91,65,75,1.68,broken clouds
9,9,Mar Del Plata,AR,-38.0023,-57.5575,55.13,62,90,4.00,overcast clouds
10,10,Albany,US,42.6001,-73.9662,65.62,83,45,5.91,scattered clouds
12,12,Mafinga,TZ,-7.2500,35.0667,69.21,51,50,2.15,scattered clouds
13,13,Chumphon,TH,10.3333,98.8333,72.43,93,100,1.66,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
import numpy as np

clean_df = preferred_cities_df.replace(' ', np.nan)
nan_values = clean_df[clean_df.isna().any(axis=1)] 
nan_values


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
215,215,Ondangwa,NaN,-17.9167,15.9500,61.45,11,15,6.89,few clouds
280,280,Rundu,NaN,-17.9333,19.7667,64.56,17,4,5.44,clear sky
324,324,Luderitz,NaN,-26.6481,15.1594,56.84,72,0,23.35,clear sky
356,356,Walvis Bay,NaN,-22.9575,14.5053,54.09,89,0,11.41,clear sky
447,447,Henties Bay,NaN,-22.1160,14.2845,53.85,92,1,8.75,clear sky
511,511,Gobabis,NaN,-22.4500,18.9667,51.64,20,0,9.22,clear sky


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna(subset = ["Country"], inplace=True)

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,82.72,light rain,22.0752,-159.3190,
3,Atuona,PF,78.28,few clouds,-9.8000,-139.0333,
4,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
5,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
6,Rikitea,PF,73.11,few clouds,-23.1203,-134.9692,
7,Hermanus,ZA,51.91,broken clouds,-34.4187,19.2345,
9,Mar Del Plata,AR,55.13,overcast clouds,-38.0023,-57.5575,
10,Albany,US,65.62,scattered clouds,42.6001,-73.9662,
12,Mafinga,TZ,69.21,scattered clouds,-7.2500,35.0667,
13,Chumphon,TH,72.43,overcast clouds,10.3333,98.8333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [57]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace("", np.nan)
clean_hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,82.72,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Atuona,PF,78.28,few clouds,-9.8000,-139.0333,Villa Enata
4,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
5,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Rikitea,PF,73.11,few clouds,-23.1203,-134.9692,People ThankYou


In [56]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [65]:
# 11a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"] 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))